# Chat Completions

In this notebook, we will explore how we can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors

In [0]:
%pip install -U openai gradio
dbutils.library.restartPython()

In [0]:
from openai import OpenAI
import os
import json
import gradio as gr

In [0]:
DATABRICKS_TOKEN = dbutils.secrets.get(scope = "db-field-eng", key = "va-pat-token")

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints"
)

#### Helper Functions
In this new helper function, we're going to pass in a list of messages

In [0]:
def get_completion_from_messages(messages, model="va_ext_azure_openai_gpt35", temperature = 0):
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature = temperature, # this sets the degree of randomness of the model's output and allows it to be more creative or less deterministic
    max_tokens=2000, # sets max number of tokens for output response
  )
  
  return completion.choices[0].message.content

#### Testing helper function

In [0]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [0]:
response = get_completion_from_messages(messages, temperature = 1) # we are using a higher temperature for creative jokes
print(response)

Each conversation with the language model is a standalone interaction so we must provide all conversation history and relevant messages as context for the model to draw from in the current conversation

In [0]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Vicky'}  ]

response = get_completion_from_messages(messages, temperature = 1)
print(response)

In [0]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

In [0]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Vicky'},
{'role':'assistant', 'content': 'Hello Vicky! It\'s nice to meet you. How can I assist you today?'},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]

response = get_completion_from_messages(messages, temperature = 1)
print(response)

#### Order Bot
We can automate the collection of user prompts and assistant responses to build an Order Bot. The OrderBot will take orders at a pizza restaurant.

In [0]:
# Context for the chatbot
context = [{'role': 'system', 'content': """
You are OrderBot, an automated service to collect orders for a fast food chain restaurant.

** You first greet the customer and show them a brief overview of mains (pizzas) in our menu

** You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. While collecting the order, you can recommend some other options on the menu (along with their sizes) that you would recommend with the items in the order.

** You then ask if it's a pickup or delivery. Finally, perform the calculations carefully step-by-step and add 10 % tax to get to the order total. Show that amount along with an order summary along with your calculations.

# Ensure to ask the address if it's a delivery. For pickups, ask the customer to collect the order from Shop-1, Hiranandani Gardens, Powai in 30 minutes.
# Make sure to clarify all options, extras, and sizes to uniquely identify the item from the menu.
# Respond with `As an order bot, I cannot assist with that` for anything that is not related to the food delivery service for your restaurant.
# You respond in a short, very conversational friendly style and do not ask multiple questions at a time or suggest multiple options at a time

The menu includes:
- pepperoni pizza  12.95, 10.00, 7.00
- cheese pizza   10.95, 9.25, 6.50
- eggplant pizza   11.95, 9.75, 6.75
- fries 4.50, 3.50
- greek salad 7.25
- Toppings:
  a. extra cheese 2.00
  b. mushrooms 1.50
  c. sausage 3.00
  d. canadian bacon 3.50
  e. AI sauce 1.50
  f. peppers 1.00
- Drinks:
  a. coke 3.00, 2.00, 1.00
  b. sprite 3.00, 2.00, 1.00
  c. bottled water 5.00
"""}]

In [0]:
# Function to process the conversation
def collect_messages(prompt, history):
  context.append({'role': 'user', 'content': prompt})
  response = get_completion_from_messages(context, model="va_ext_azure_openai", temperature=0)
  context.append({'role': 'assistant', 'content': response})
  return response

In [0]:
demo = gr.ChatInterface(
    collect_messages,
    type="messages",
    textbox=gr.Textbox(placeholder="I would like to order something today", container=False, scale=7),
    title="Order Bot",
    description="This chatbot is a demo example of an order bot for a fast food chain",
    cache_examples=False,
)

demo.launch(share=True, debug=False)

Finally we can create an order JSON summary to send to our order system. We'll use temperature = 0 here as we want these tasks to be very predictable to avoid downstream failures

In [0]:
prompt = [{'role':'user', 'content':f"""create a json summary of the previous food order, delimited by ```. Extract the following information from the order summary:
1) pizza, include size
2) list of toppings
3) list of drinks, include size
4) list of sides include size
5) total price 

Order summary:
```{context}```

Ensure to add pricing for each line item. Only have final numbers and don't include expressions for prices

Example output:
```
{{"pizza": {{"type": "cheese", "size": "large", "toppings": ["mushrooms", "extra cheese"], "price": 13.95}}, "drinks": [{{"type": "coke", "size": "medium", "price": 2.00}}], "sides": [{{"type": "fries", "size": "medium", "price": 3.50}}], "base_price": 20.45, "tax_perc": 10.00, "total_price": 22.495}}```
"""}]

response = get_completion_from_messages(prompt, model="databricks-meta-llama-3-1-70b-instruct", temperature = 0)
print(response)